# Prediction on Fresh Feedbacks

## Importing necessary Libraries

In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
import pickle
import xgboost as xgb
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91626\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Load the new CSV file

predict_data = pd.read_csv('SentimentBulk.csv')

In [4]:
predict_data.head()

,Sentence
0,Love my Echo!
1,Loved it!
2,"Sometimes while playing a game, you can answer..."
3,I have had a lot of fun with this thing. My 4 ...
4,This item did not work. Certified refurbished ...


In [5]:
reviews = predict_data['Sentence']

## Preprocessing

In [17]:
# Preprocess the text data

corpus = []
ps = PorterStemmer()

for review in reviews:
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
# Load the CountVectorizer

cv = pickle.load(open('Models/countVectorizer.pkl', 'rb'))

In [19]:
# Load the trained model

model_xgb = pickle.load(open('Models/model_xgb.pkl', 'rb'))

In [20]:
# Transform the text data

X_predict = cv.transform(corpus).toarray()

## Predictions

In [22]:
# Make predictions

predictions = model_xgb.predict(X_predict)
print(f'predictions:', predictions)

predictions: [1 1 1 1 0 0 0]


In [23]:
# Map predictions to 'positive' and 'negative'

predict_data['predictions'] = predictions
predict_data['predictions'] = predict_data['predictions'].map({1: 'positive', 0: 'negative'})

In [24]:
predict_data.head()

,Sentence,predictions
0,Love my Echo!,positive
1,Loved it!,positive
2,"Sometimes while playing a game, you can answer...",positive
3,I have had a lot of fun with this thing. My 4 ...,positive
4,This item did not work. Certified refurbished ...,negative


In [26]:
# Save the DataFrame to a new CSV file

predict_data.to_csv('predicted_reviews.csv', index=False)